In [1]:
import numpy as np
import nibabel as nib
import dipy.reconst.dti as dti
from dipy.data import fetch_stanford_hardi, read_stanford_hardi
from dipy.reconst.dti import quantize_evecs
from dipy.data import get_sphere
from dipy.reconst.dti import fractional_anisotropy, color_fa, lower_triangular
from dipy.segment.mask import median_otsu
from dipy.tracking.eudx import EuDX
from dipy.io.trackvis import save_trk


/home/jyeatman/anaconda/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [5]:
fetch_stanford_hardi()

Creating new directory /home/jyeatman/.dipy/stanford_hardi
Done.
Files copied in folder /home/jyeatman/.dipy/stanford_hardi


In [6]:
img, gtab = read_stanford_hardi()
data = img.get_data()

In [7]:
maskdata, mask = median_otsu(data, 3, 1, True,
                             vol_idx=range(10, 50), dilate=2)

In [8]:
tenmodel = dti.TensorModel(gtab)
tenfit = tenmodel.fit(maskdata)

In [9]:
FA = fractional_anisotropy(tenfit.evals)
FA[np.isnan(FA)] = 0
evecs = tenfit.evecs
fa_img = nib.Nifti1Image(FA.astype(np.float32), img.get_affine())

In [10]:
sphere = get_sphere('symmetric724')
peak_indices = quantize_evecs(evecs, sphere.vertices)
eu = EuDX(FA.astype('f8'), peak_indices, seeds=5000000, odf_vertices = sphere.vertices, a_low=0.2)

tensor_streamlines = [streamline for streamline in eu]

/home/jyeatman/git/dipy/dipy/reconst/dti.py:1802: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if odf_vertices == None:


In [14]:
#save_trk("./tensor_streamlines.trk", tensor_streamlines, fa_img.get_affine(), FA.shape)

In [11]:
len(tensor_streamlines)

977354

In [1]:
tensor_streamlines[1000]

NameError: name 'tensor_streamlines' is not defined

In [12]:
hdr = nib.trackvis.empty_header()
hdr['voxel_size'] = img.get_header().get_zooms()[:3]
hdr['voxel_order'] = 'RAS'
hdr['dim'] = FA.shape
hdr['vox_to_ras'] = img.get_affine()
tensor_streamlines_trk = ((sl, None, None) for sl in tensor_streamlines)
ten_sl_fname = 'tensor_streamlines.trk'
nib.trackvis.write(ten_sl_fname, tensor_streamlines_trk, hdr, points_space='rasmm')

In [13]:
hdr

array(('TRACK', [71, 87, 62], [2.0, 2.0, 2.0], [0.0, 0.0, 0.0], 0, ['', '', '', '', '', '', '', '', '', ''], 0, ['', '', '', '', '', '', '', '', '', ''], [[2.0, 0.0, 0.0, -80.0], [0.0, 2.0, 0.0, -120.0], [0.0, 0.0, 2.0, -60.0], [0.0, 0.0, 0.0, 1.0]], '', 'RAS', '', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], '', '', '', '', '', '', '', 0, 2, 1000), 
      dtype=[('id_string', 'S6'), ('dim', '<i2', (3,)), ('voxel_size', '<f4', (3,)), ('origin', '<f4', (3,)), ('n_scalars', '<i2'), ('scalar_name', 'S20', (10,)), ('n_properties', '<i2'), ('property_name', 'S20', (10,)), ('vox_to_ras', '<f4', (4, 4)), ('reserved', 'S444'), ('voxel_order', 'S4'), ('pad2', 'S4'), ('image_orientation_patient', '<f4', (6,)), ('pad1', 'S2'), ('invert_x', 'S1'), ('invert_y', 'S1'), ('invert_z', 'S1'), ('swap_xy', 'S1'), ('swap_yz', 'S1'), ('swap_zx', 'S1'), ('n_count', '<i4'), ('version', '<i4'), ('hdr_size', '<i4')])

In [14]:
len(tensor_streamlines)

977354

In [12]:
tensor_streamlines[0]

array([[ 26.4841423 ,  38.62833023,  55.76271439],
       [ 26.57774162,  38.49172211,  55.29093552],
       [ 26.64459229,  38.38281631,  54.80754089],
       [ 26.67384911,  38.28636551,  54.31780243],
       [ 26.67930412,  38.199646  ,  53.82541275],
       [ 26.65811348,  38.11829758,  53.33252716],
       [ 26.60972404,  38.04495239,  52.84030914],
       [ 26.54303741,  37.99626923,  52.3471756 ],
       [ 26.45503044,  37.96047974,  51.8562851 ],
       [ 26.32678413,  37.92443085,  51.37435532],
       [ 26.16465759,  37.89036942,  50.90259933],
       [ 25.99437714,  37.88658905,  50.43250275],
       [ 25.81068993,  37.91730118,  49.96848297],
       [ 25.60430717,  37.9600296 ,  49.51507187],
       [ 25.38165474,  38.02501678,  49.07212448],
       [ 25.16344643,  38.09794235,  48.62820435],
       [ 24.94886398,  38.17311859,  48.18289185],
       [ 24.7429409 ,  38.25121307,  47.73400497],
       [ 24.54547119,  38.33713531,  47.28276062],
       [ 24.36642075,  38.44501